In [ ]:
# Import library

# Packages required
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
from astropy.io import fits
import os
from amiral import instructment, utils, parameter, gradient, minimisation, array
from scipy.optimize import minimize 
%matplotlib inline
from deconvbench import Deconvbench
from mpl_toolkits.axes_grid1 import make_axes_locatable

import tools

In [ ]:
# Function for getting snr
def get_snr (array, noise):
    
    mean = np.mean(array)
    sig2 = np.std(noise)
    
    snr = mean / sig2
    
    return snr

def psd_object (param):
    
    rho = np.fft.fftshift(utils.dist(512))/param[1]
    psd_obj =  param[0]/ (np.power(rho,param[2]) + 1.)
    
    return psd_obj
    
def plot_psd_object(psd_obj): 
    
    fig, ax = plt.subplots()
    ycent = int((256*aosys_cls.samp_factor[0])//2)

    ax.plot(np.abs(psd_obj[ycent,...]))
    ax.set_title('PSD Object(total)')
    ax.axhline(y=1, color = 'r', ls = '--')
    
    pass


def forced_zero (array):

    """
    Checking the input image array and make sure there is no zero in the input array 
    Since the input array is treated as an object, val < 0 / = nan would result in error 
    while estimating the photon noise of your simulated observations


    Args:
        array ([type]): [description]

    Returns:
        [type]: [description]
    """
    
    ind = np.where(array < 0)

    if ((len(ind))) != 0: 
        array[ind] = 0.
        return array

    else: 
        return array
    
    
def create_psfao19_otf (otf_tel, guess, aosys_cls): 
    
    # Use PSFAO19 model to create a PSF
    psd_ao = aosys_cls.psd_residual_ao (guess = guess)
    psd_halo = aosys_cls.psd_residual_halo(r0 = guess[0])
    
    psd = psd_ao + psd_halo

    otf_atmo = aosys_cls.otf_atmo(psd)
    otf_total = otf_atmo*otf_tel
    
    
    return otf_atmo,otf_total

In [ ]:
# PATH

# Parameters for the PSF
psf_dict = {
    "r0": 0.15,                  
    "background": 0.,      
    "amplitude": 1.5,       
    "ax": 0.05,                            
    "beta": 1.5, 
    "mu": 0., 
    "rho0": 0., 
    "p": 0. 
}

# Variable
RON = 10. # CCD read-out noise standard-deviation [e-]
FLUX = np.linspace(5e6,5e9,10)
DIMENSION = int(720*3)

flux_snr = FLUX[9]
print(flux_snr)

# psf_dict["r0"] = 0.1
# psf_dict["amplitude"] = 0.5

Get the object

In [ ]:
path_galaxy = '/Users/alau/Data/mock_sloan_data/Sbc_300_g_0.fits'
_gal = fits.open(path_galaxy)
gal = _gal[0].data


dim = gal.shape[0]

print(dim)
gal_resize = np.zeros((dim*3,dim*3))


cuta = int(dim*3//2-(dim/2))
cutb = int(dim*3//2+(dim/2))

print(cuta)
 
gal_resize[cuta:cutb,cuta:cutb] = gal

# Calibrating the flux
gal_resize = forced_zero(gal_resize)
gal_resize = gal_resize/np.sum(gal_resize)*flux_snr


img = gal_resize

In [ ]:
rcParams['figure.figsize'] = 33 ,24

fig, ax = plt.subplots(1,2)
fig.tight_layout(pad=0.4, w_pad=0.5, h_pad=2.0)

im = ax[0].imshow(np.real(gal))
ax[0].set_title('Image')
fig.colorbar(im, ax=ax[0])

im1 = ax[1].imshow(np.real(img))
ax[1].set_title('Estimated Object')
fig.colorbar(im1, ax=ax[1])


We need a PSF

In [ ]:
# Set up the telescope and produce a PSF
amiral_dict = {
    "r0": 0.20,                  
    "background": 0.,      
    "amplitude": 1.5,       
    "ax": 0.05,                            
    "beta": 1.5, 
    "mu": 0., 
    "rho0": 0., 
    "p": 0. 
}

aosys_dict = {
    'diameter': 8 , 
    'occ_ratio': 0.14, 
    'no_acutuator' : 20, 
    'wavelength': 500, 
    'dimension': 720,
    'resolution_rad' : 2.083e-8
}

# Passing parametpsd_arrayers from the telesope to the aosystem
aosys_cls = instructment.aoSystem( 
        diameter = aosys_dict['diameter'], occ_ratio = aosys_dict['occ_ratio'], 
        no_acutuator= aosys_dict['no_acutuator'], wavelength = aosys_dict['wavelength']*1e-9, 
        resolution_rad = aosys_dict['resolution_rad'], 
        dimension=aosys_dict['dimension'])  


psf_keys, psf_param = utils.dict2array(psf_dict)
amiral_keys, psf_guess = utils.dict2array(amiral_dict)

In [ ]:
psd_ao = aosys_cls.psd_residual_ao (psf_param)
psd_halo = aosys_cls.psd_residual_halo(psf_dict['r0'])

psd = psd_halo + psd_ao 

pupil = aosys_cls.get_pupil_plane()
otf_tel = aosys_cls.pupil_to_otf_tel(pupil)
psf_tel = np.fft.ifft2(np.fft.fftshift(otf_tel))

integral, SR = aosys_cls.psd_integral(psd_ao, psf_dict['r0'])

otf_atmo = aosys_cls.otf_atmo(psd)

otf_total = np.fft.ifftshift(otf_atmo * otf_tel)

psf_total = aosys_cls.psfao(otf_total)
print(np.shape(psf_total))

plt.imshow(np.log10(np.real(psf_tel)))

In [ ]:
# Noise 
gauss_noise = np.random.randn(np.shape(gal_resize)[0],np.shape(gal_resize)[0])*RON

rng = np.random.default_rng()
photon_noise = rng.poisson(gal_resize)

noise = gauss_noise + photon_noise

In [ ]:
ft_obj = np.fft.fft2(np.fft.ifftshift(gal_resize))

ft_image = ft_obj*otf_total

_obj = np.real(np.fft.ifft2(ft_image))

_obj = forced_zero(_obj)


noise = photon_noise + gauss_noise

empty_arr = np.zeros((DIMENSION,DIMENSION))+1.

psd_noise = np.average(ft_image)*empty_arr
psd_asteroid = np.abs(ft_image)**2

snr = psd_asteroid/psd_noise

est_obj = tools.wiener_filter(ft_image,otf_total,snr)

# obs_image = noise + _obj
obs_image = _obj

img = obs_image

plt.imshow(np.real(np.log10(otf_total)))

In [ ]:
# snr_scalar = get_snr(ft_img_noise**2, tools.fft2D(noise)**2)
# print(snr_scalar)

# observed_noisy_image = tools.ifft2D(ft_img_noise, norm = True)

# plt.imshow(np.real(observed_noisy_image))
print("Sum: ", np.sum(gal_resize))
print("Sum: ", np.sum(obs_image))
print("Noise of the object: ", np.sum(noise))
print("Retrieved Flux: ",np.sum(obs_image) - np.sum(noise))

In [ ]:
# empty_arr = np.zeros((DIMENSION,DIMENSION))+1.

# psd_noise = np.average(observed_noisy_image)*empty_arr
# psd_asteroid = np.abs(ft_img_noise)**2

# snr = psd_asteroid/psd_noise


# est_obj = tools.wiener_filter(ft_img_noise,otf_total,snr)

In [ ]:
# fig, ax = plt.subplots(1,2)

# rcParams['figure.figsize'] = 33 ,24

# im = ax[0].imshow(np.real(est_obj))
# fig.colorbar(im, ax=ax[0])
# ax[0].set_title('Estimated Object')

# im1 = ax[1].imshow(np.real(observed_noisy_image))
# fig.colorbar(im1, ax=ax[1])
# ax[1].set_title('Observed Image')


In [ ]:
# print("\nFlux of the object: ", np.sum(asteriod_resize))
# print("\nNoise ofthe object: ", np.sum(np.abs(noise)))
# print("\nSNR: ", np.sum(asteriod_resize)/np.sum(np.abs(noise)))

# print(snr_scalar)

In [ ]:
# print("Sum: ", np.sum(est_obj))
# print("Noise of the object: ", np.sum(noise))
# print("Retrieved Flux: ",np.sum(est_obj) - np.sum(noise))

In [ ]:
# img = np.real(np.fft.fftshift(observed_noisy_image))

# _img = fits.open("/Users/alau/Data/amiral_fits/VESTA/SNR/case_0/VESTA_84.fits")
# img = _img[0].data

amiralparam = parameter.amiralParam(img ,guess = psf_guess, aosys = aosys_cls)

In [ ]:
# What variables to be minimised
param_mask = np.asarray([1,1,0,0,0])
hyper_param_mask = np.asarray([1,1,1])

mask = np.concatenate((param_mask,hyper_param_mask))

hyper_guess = amiralparam.hyperparam_initial(psf_guess, debug = True)
hyper_min, hyper_max = amiralparam.hyperparam_bound(psf_guess, p_upperbound = 100., debug = True)

psf_guess[-3] = hyper_guess[0] 
psf_guess[-2] = hyper_guess[1] 
psf_guess[-1] = hyper_guess[2] 

param_min = np.asarray([0.05,0,0,1e-8,1.01])
param_max =  np.asarray([1.,1e8,1e8,1e3,10])

upperbound = np.concatenate((param_max, hyper_max))
lowerbound = np.concatenate((param_min, hyper_min))

param_numerical_condition = np.array([1., 1e-4, 1., 1., 1.])
hyperparam_numerical_condition = np.array([hyper_guess[0], hyper_guess[1], 1.])

numerical_condition = np.concatenate((param_numerical_condition, hyperparam_numerical_condition))

amiral_cls = parameter.amiral(img=img, guess=psf_guess, aosys = aosys_cls, upperbound = upperbound, lowerbound= lowerbound, numerical_condition = numerical_condition, fourier_variable = amiralparam.fourier_variable, mask = mask)

In [ ]:
est_criterion, value_criterion, value_grad = amiral_cls.minimisation(psf_guess)

In [ ]:
print(est_criterion)

In [ ]:
est_otf_atmo, est_otf = create_psfao19_otf(otf_tel, est_criterion[0:5], aosys_cls)
est_psf = np.fft.ifft2(np.fft.ifftshift(est_otf))

In [ ]:
est_SR = np.max(np.real(est_psf)) / np.max(np.real(psf_tel))
SR = np.max(psf_total) / np.max(psf_tel)

# Plot the PSF slice
rcParams['figure.figsize'] = 13 ,11

ycent = int((psf_total.shape[0]//2))
    
fig, ax = plt.subplots()

ax.set_title(r"PSF Estimation (without noise)$\mathrm{ \{r_0, sig^2, mu, \rho_0 \}}$",fontsize = 14)
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_ylabel(r'$\mathrm{Mean Intensity [e^-]}$', fontsize = 14)
ax.set_xlabel(r'$\mathrm{Position [pixel]}$', fontsize = 14)
ax.plot(utils.mean_cir_array(np.real(psf_total)), label = "True")
ax.plot(utils.mean_cir_array(np.real(est_psf)), label = "Estimated PSF")
ax.legend()



In [ ]:
np.real(est_SR-SR)*100

In [ ]:
# dec = Deconvbench(img,est_psf,ron = 10)
# dec.verbose_modulo = 10 # print every 10 iteration
# dec.regularization.scale *= 4. # sharpen details (reduce regularization)
# estim = dec.run()

In [ ]:
# estim_norm = estim/gal_resize.max()
# gal_resize_norm = gal_resize/gal_resize.max()


# fig, ax = plt.subplots(1,2)

# rcParams['figure.figsize'] = 33 ,24

# im = ax[0].imshow(np.abs(estim_norm-gal_resize_norm))
# fig.colorbar(im, ax=ax[0])
# ax[0].set_title('Residual (Normalised)\nDiff(SR) = 2.16%')

# im1 = ax[1].imshow(np.abs(gal_resize-estim))
# fig.colorbar(im1, ax=ax[1])
# ax[1].set_title('Residual\nDiff(SR) = 2.16%')

In [ ]:
# dec = Deconvbench(img,psf_total,ron = 10)
# dec.verbose_modulo = 10 # print every 10 iteration
# dec.regularization.scale *= 4. # sharpen details (reduce regularization)
# estim_1 = dec.run()